In [3]:
import os
import sys

In [4]:
ROOT = os.path.join(os.pardir)
MODEL_DIR = os.path.join(ROOT, 'models')
MODEL_WEIGHTS_DIR = os.path.join(MODEL_DIR, 'model_weights')

sys.path.append(os.path.join(ROOT, "src"))

%load_ext autoreload
%autoreload 2

In [5]:
IMG_SIZE = 224
N_CLASSES = 196
LN_RATE = 0.001

# Create Model

## ResNet50

In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import (
    Dense, Activation, Flatten, Dropout, BatchNormalization
)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

In [13]:
base_model = ResNet50(
    weights='imagenet', 
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='avg'
)

In [14]:
base_model.input

<tf.Tensor 'input_2:0' shape=(?, 224, 224, 3) dtype=float32>

In [15]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

**Freezing layers**

In [16]:
# Freeze pretrained layers
for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output

x = Flatten()(x)

x = BatchNormalization()(x)
x = Dropout(rate=0.25)(x)
x = Dense(512, activation='relu')(x)

x = BatchNormalization()(x)
x = Dropout(rate=0.5)(x)
predictions = Dense(N_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

**Loading Weights**

In [25]:
model.load_weights(os.path.join(MODEL_WEIGHTS_DIR, 'model.h5'))

In [26]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(lr=LN_RATE),
    # what about precision and recall?
    metrics=['acc', Precision(), Recall()]
)

What metrics to keep track of?

In [40]:
model.save(os.path.join(MODEL_DIR, 'model.h5'))

In [39]:
model.save_weights(os.path.join(MODEL_DIR, 'model.h5'))